## ICR - Identifying Age-Related Conditions¶


Use Machine Learning to detect conditions with measurements of anonymous characteristics
https://www.kaggle.com/code/samuelabatnehendalie/icr-identifying-age-related-conditions

### Importing Libraries

In [184]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [185]:
''' use this code when upload to kaggle
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
'''

' use this code when upload to kaggle\n# This Python 3 environment comes with many helpful analytics libraries installed\n# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python\n# For example, here\'s several helpful packages to load\n\nimport numpy as np # linear algebra\nimport pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)\nimport matplotlib.pyplot as plt\nimport seaborn as sns\n\n# Input data files are available in the read-only "../input/" directory\n# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory\n\nimport os\nfor dirname, _, filenames in os.walk(\'/kaggle/input\'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n\n# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"\n# You can also write temporary files to /kaggle/temp/, but they wo

In [186]:
directory = "../data"


In [187]:
train = pd.read_csv(os.path.join(directory, "train.csv"))
meta_data = pd.read_csv(os.path.join(directory, "greeks.csv"))
test = pd.read_csv(os.path.join(directory, "test.csv"))

### Exploratory Data Analysis

In [188]:
train.head() # a lot of columns!

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


In [189]:
train.describe() # may be coming from different distribution


,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
count,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,...,616.000000,617.000000,615.000000,617.000000,617.000000,617.000000,617.000000,617.000000,616.000000,617.000000
mean,0.477149,3502.013221,118.624513,38.968552,10.128242,5.545576,0.060320,10.566447,8.053012,5350.388655,...,5.433199,3.533905,0.421501,20.724856,131.714987,14679.595398,31.489716,50.584437,8.530961,0.175041
std,0.468388,2300.322717,127.838950,69.728226,10.518877,2.551696,0.416817,4.350645,65.166943,3021.326641,...,11.496257,50.181948,1.305365,9.991907,144.181524,19352.959387,9.864239,36.266251,10.327010,0.380310
min,0.081187,192.593280,85.200147,3.177522,8.138688,0.699861,0.025578,3.396778,1.229900,1693.624320,...,0.173229,0.497060,0.067730,4.102182,72.611063,13.038894,9.432735,0.897628,0.001129,0.000000
25%,0.252107,2197.345480,85.200147,12.270314,8.138688,4.128294,0.025578,8.129580,1.229900,4155.702870,...,0.173229,0.497060,0.067730,14.036718,72.611063,2798.992584,25.034888,23.011684,0.124392,0.000000
50%,0.354659,3120.318960,85.200147,20.533110,8.138688,5.031912,0.025578,10.461320,1.229900,4997.960730,...,3.028141,1.131000,0.250601,18.771436,72.611063,7838.273610,30.608946,41.007968,0.337827,0.000000
75%,0.559763,4361.637390,113.739540,39.139886,8.138688,6.431634,0.036845,12.969516,5.081244,6035.885700,...,6.238814,1.512060,0.535067,25.608406,127.591671,19035.709240,36.863947,67.931664,21.978000,0.000000
max,6.161666,28688.187660,1910.123198,630.518230,178.943634,38.270880,10.315851,38.971568,1463.693448,53060.599240,...,137.932739,1244.227020,31.365763,135.781294,1497.351958,143790.071200,81.210825,191.194764,21.978000,1.000000


In [190]:
train.info() # can see that there are several null values in the data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617 entries, 0 to 616
Data columns (total 58 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Id      617 non-null    object 
 1   AB      617 non-null    float64
 2   AF      617 non-null    float64
 3   AH      617 non-null    float64
 4   AM      617 non-null    float64
 5   AR      617 non-null    float64
 6   AX      617 non-null    float64
 7   AY      617 non-null    float64
 8   AZ      617 non-null    float64
 9   BC      617 non-null    float64
 10  BD      617 non-null    float64
 11  BN      617 non-null    float64
 12  BP      617 non-null    float64
 13  BQ      557 non-null    float64
 14  BR      617 non-null    float64
 15  BZ      617 non-null    float64
 16  CB      615 non-null    float64
 17  CC      614 non-null    float64
 18  CD      617 non-null    float64
 19  CF      617 non-null    float64
 20  CH      617 non-null    float64
 21  CL      617 non-null    float64
 22  CR

In [191]:
train.nunique()

Id       617
AB       217
AF       599
AH       227
AM       605
AR       130
AX       427
AY       148
AZ       484
BC       259
BD       617
BN        53
BP       612
BQ       515
BR       566
BZ       115
CB       553
CC       602
CD       584
CF       586
CH       135
CL       123
CR       595
CS       576
CU       307
CW       426
DA       611
DE       616
DF       137
DH       191
DI       571
DL       604
DN       576
DU       253
DV        39
DY       590
EB       439
EE       513
EG       610
EH       127
EJ         2
EL       311
EP       275
EU       455
FC       600
FD       337
FE       615
FI       498
FL       388
FR       435
FS       161
GB       560
GE       264
GF       611
GH       596
GI       615
GL       355
Class      2
dtype: int64

In [192]:
null_cols = train.columns[train.isnull().any()]
print(train[null_cols].isnull().sum()) # remove BQ and EL since they have more 50 null values


BQ    60
CB     2
CC     3
DU     1
EL    60
FC     1
FL     1
FS     2
GL     1
dtype: int64


In [193]:
train = train.drop(['BQ', 'EL'], axis=1)



In [194]:
null_cols = train.columns[train.isnull().any()]
for col in null_cols:
    train[col].fillna(train[col].mean(), inplace=True)


In [195]:
train.info() # no more null values


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617 entries, 0 to 616
Data columns (total 56 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Id      617 non-null    object 
 1   AB      617 non-null    float64
 2   AF      617 non-null    float64
 3   AH      617 non-null    float64
 4   AM      617 non-null    float64
 5   AR      617 non-null    float64
 6   AX      617 non-null    float64
 7   AY      617 non-null    float64
 8   AZ      617 non-null    float64
 9   BC      617 non-null    float64
 10  BD      617 non-null    float64
 11  BN      617 non-null    float64
 12  BP      617 non-null    float64
 13  BR      617 non-null    float64
 14  BZ      617 non-null    float64
 15  CB      617 non-null    float64
 16  CC      617 non-null    float64
 17  CD      617 non-null    float64
 18  CF      617 non-null    float64
 19  CH      617 non-null    float64
 20  CL      617 non-null    float64
 21  CR      617 non-null    float64
 22  CS

In [196]:
ytrain = train['Class']
print(ytrain)
train = train.drop(['Class', 'Id'], axis=1)
train.head()

0      1
1      0
2      0
3      0
4      1
      ..
612    0
613    0
614    0
615    0
616    0
Name: Class, Length: 617, dtype: int64


,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
0,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,...,3.583450,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343
1,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,...,10.358927,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000
2,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,...,11.626917,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941
3,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,...,14.852022,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829
4,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,...,13.666727,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614


In [197]:
train['EJ'].unique()

array(['B', 'A'], dtype=object)

In [198]:
# make categorical data into numerical data
train.EJ = train.EJ.replace({'A': 0, 'B': 1})
train.head()


,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
0,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,...,3.583450,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343
1,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,...,10.358927,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000
2,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,...,11.626917,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941
3,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,...,14.852022,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829
4,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,...,13.666727,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614


In [206]:
 # referenced https://www.kaggle.com/code/dan3dewey/icr-2023-balanced-log-loss
def competition_log_loss(y_true, y_pred): # use predict proba
    # y_true: correct labels 0, 1
    # y_pred: predicted probabilities of class=1
    # Implements the Evaluation equation with w_0 = w_1 = 1.
    # Calculate the number of observations for each class
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    print(y_pred.shape)
    # Calculate the predicted probabilities for each class
    p_1 = np.clip(y_pred[:,1], 1e-15, 1 - 1e-15)
    p_0 = 1 - p_1
    # Calculate the average log loss for each class
    #print("reached")
    print(p_0.shape, p_1.shape, y_true.shape)

    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0)) / N_0
    log_loss_1 = -np.sum(y_true * np.log(p_1)) / N_1
    # return the (not further weighted) average of the averages
    return (log_loss_0 + log_loss_1)/2


def competition_log_loss2(y_true, y_pred): # for grid search
    # y_true: correct labels 0, 1
    # y_pred: predicted probabilities of class=1
    # Implements the Evaluation equation with w_0 = w_1 = 1.
    # Calculate the number of observations for each class
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    print(y_pred.shape)
    # Calculate the predicted probabilities for each class
    p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)
    p_0 = 1 - p_1
    # Calculate the average log loss for each class
    #print("reached")
    print(p_0.shape, p_1.shape, y_true.shape)

    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0)) / N_0
    log_loss_1 = -np.sum(y_true * np.log(p_1)) / N_1
    # return the (not further weighted) average of the averages
    return (log_loss_0 + log_loss_1)/2

### Train Test Split

In [207]:
# train test split such that same distribution of classes in train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, ytrain, test_size=0.2, random_state=42, stratify=ytrain)

/Users/gary/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1609: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
/Users/gary/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1609: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review 

In [218]:
# xgboost model
from xgboost import XGBClassifier

xgb = XGBClassifier(
       learning_rate=0.01,
      max_depth=7, n_estimators=300)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_train)

print('Accuracy of XGB classifier on train set: {:.2f}'.format(accuracy_score(y_train, y_pred)))
print(confusion_matrix(y_train, y_pred))
print(classification_report(y_train, y_pred))

y_pred = xgb.predict(X_test)
y_pred2 =   xgb.predict_proba(X_test)

print('Accuracy of XGB classifier on test set: {:.2f}'.format(accuracy_score(y_test, y_pred)))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

print(competition_log_loss(y_test, y_pred2))







Accuracy of XGB classifier on train set: 1.00
[[407   0]
 [  2  84]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       407
           1       1.00      0.98      0.99        86

   micro avg       1.00      1.00      1.00       493
   macro avg       1.00      0.99      0.99       493
weighted avg       1.00      1.00      1.00       493

Accuracy of XGB classifier on test set: 0.90
[[96  6]
 [ 7 15]]
              precision    recall  f1-score   support

           0       0.93      0.94      0.94       102
           1       0.71      0.68      0.70        22

   micro avg       0.90      0.90      0.90       124
   macro avg       0.82      0.81      0.82       124
weighted avg       0.89      0.90      0.89       124

(124, 2)
(124,) (124,) (124,)
0.46885364824979037


In [222]:
import optuna
from xgboost import XGBClassifier

def objective(trial):
    # Define the search space for hyperparameters
    param_space = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 7),
        'n_estimators': trial.suggest_categorical('n_estimators', [100, 200, 300])
    }

    # Create an XGBoost classifier with the suggested hyperparameters
    xgb_model = XGBClassifier(**param_space)

    # Train and evaluate the model
    xgb_model.fit(X_train, y_train)
    y_pred = xgb_model.predict_proba(X_test)
    score = competition_log_loss(y_test, y_pred)

    return score

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)
best_params = study.best_params



[I 2023-05-15 20:22:40,650] A new study created in memory with name: no-name-7fa3a00c-ea2b-4b2e-87eb-63ff220c10b3
/Users/gary/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  
[I 2023-05-15 20:22:41,747] Trial 0 finished with value: 0.6185383705850591 and parameters: {'learning_rate': 0.0019361280899354814, 'max_depth': 4, 'n_estimators': 100}. Best is trial 0 with value: 0.6185383705850591.


(124, 2)
(124,) (124,) (124,)


[I 2023-05-15 20:22:43,296] Trial 1 finished with value: 0.5552209155250041 and parameters: {'learning_rate': 0.025816701364335733, 'max_depth': 4, 'n_estimators': 300}. Best is trial 1 with value: 0.5552209155250041.


(124, 2)
(124,) (124,) (124,)


[I 2023-05-15 20:22:44,209] Trial 2 finished with value: 0.6076737428997077 and parameters: {'learning_rate': 0.0015091219393355633, 'max_depth': 3, 'n_estimators': 200}. Best is trial 1 with value: 0.5552209155250041.


...

(124, 2)
(124,) (124,) (124,)


[I 2023-05-15 20:24:25,678] Trial 98 finished with value: 0.46738089433142427 and parameters: {'learning_rate': 0.013887677700319769, 'max_depth': 7, 'n_estimators': 100}. Best is trial 33 with value: 0.45975436175942636.


(124, 2)
(124,) (124,) (124,)


[I 2023-05-15 20:24:26,520] Trial 99 finished with value: 0.4601850212549993 and parameters: {'learning_rate': 0.016495364925348022, 'max_depth': 7, 'n_estimators': 100}. Best is trial 33 with value: 0.45975436175942636.


(124, 2)
(124,) (124,) (124,)


In [224]:
print(best_params)
print(study.best_value)

{'learning_rate': 0.01596255039414685, 'max_depth': 7, 'n_estimators': 100}
0.45975436175942636


In [176]:
# LIGHTGBM
import lightgbm as lgb

lgbm = lgb.LGBMClassifier()
lgbm.fit(X_train, y_train)
y_pred = lgbm.predict(X_train)

print('Accuracy of LGBM classifier on train set: {:.2f}'.format(accuracy_score(y_train, y_pred)))
print(confusion_matrix(y_train, y_pred))
print(classification_report(y_train, y_pred))

y_pred = lgbm.predict(X_test)
y_pred2 =   lgbm.predict_proba(X_test)

print('Accuracy of LGBM classifier on test set: {:.2f}'.format(accuracy_score(y_test, y_pred)))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

print(competition_log_loss(y_test, y_pred2))


Accuracy of LGBM classifier on train set: 1.00
[[407   0]
 [  0  86]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       407
           1       1.00      1.00      1.00        86

   micro avg       1.00      1.00      1.00       493
   macro avg       1.00      1.00      1.00       493
weighted avg       1.00      1.00      1.00       493

Accuracy of LGBM classifier on test set: 0.91
[[98  4]
 [ 7 15]]
              precision    recall  f1-score   support

           0       0.93      0.96      0.95       102
           1       0.79      0.68      0.73        22

   micro avg       0.91      0.91      0.91       124
   macro avg       0.86      0.82      0.84       124
weighted avg       0.91      0.91      0.91       124

(124, 2)
(124,) (124,) (124,)
0.6513792546428051


In [225]:
# optimize lightgbm
def objective(trial):
    # Define the search space for hyperparameters
    param_space = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 7),
        'n_estimators': trial.suggest_categorical('n_estimators', [100, 200, 300])
    }

    # Create an XGBoost classifier with the suggested hyperparameters
    lgbm_model = lgb.LGBMClassifier(**param_space)

    # Train and evaluate the model
    lgbm_model.fit(X_train, y_train)
    y_pred = lgbm_model.predict_proba(X_test)
    score = competition_log_loss(y_test, y_pred)

    return score

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)
best_params = study.best_params



[I 2023-05-15 20:29:54,943] A new study created in memory with name: no-name-2cfdd93a-5961-42d9-9755-2415ca36a1bb
/Users/gary/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  """
[I 2023-05-15 20:29:56,201] Trial 0 finished with value: 0.7709744446393468 and parameters: {'learning_rate': 0.002840513627856833, 'max_depth': 5, 'n_estimators': 100}. Best is trial 0 with value: 0.7709744446393468.


(124, 2)
(124,) (124,) (124,)


[I 2023-05-15 20:29:59,672] Trial 1 finished with value: 0.5315303193439643 and parameters: {'learning_rate': 0.006038523629475501, 'max_depth': 7, 'n_estimators': 300}. Best is trial 1 with value: 0.5315303193439643.


(124, 2)
(124,) (124,) (124,)


[I 2023-05-15 20:30:00,535] Trial 2 finished with value: 0.5043814626443425 and parameters: {'learning_rate': 0.055830187758076494, 'max_depth': 4, 'n_estimators': 100}. Best is trial 2 with value: 0.5043814626443425.


...

(124, 2)
(124,) (124,) (124,)


[I 2023-05-15 20:31:53,506] Trial 98 finished with value: 0.507648329438622 and parameters: {'learning_rate': 0.0472956855871066, 'max_depth': 4, 'n_estimators': 100}. Best is trial 74 with value: 0.48372759400721.


(124, 2)
(124,) (124,) (124,)


[I 2023-05-15 20:31:54,056] Trial 99 finished with value: 0.557295337764022 and parameters: {'learning_rate': 0.02440188585065466, 'max_depth': 3, 'n_estimators': 100}. Best is trial 74 with value: 0.48372759400721.


(124, 2)
(124,) (124,) (124,)


In [226]:
print(best_params)
print(study.best_value)


{'learning_rate': 0.05679604223386096, 'max_depth': 4, 'n_estimators': 100}
0.48372759400721


In [227]:
# ensemble
from sklearn.ensemble import VotingClassifier

xgb = XGBClassifier(learning_rate=0.015962550394, max_depth=7, n_estimators=100)
lgbm = lgb.LGBMClassifier(learning_rate=0.05679604, max_depth=4, n_estimators=100)

estimators = [('xgb', xgb), ('lgbm', lgbm)]

ensemble = VotingClassifier(estimators, voting='soft')
ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_train)

print('Accuracy of ensemble classifier on train set: {:.2f}'.format(accuracy_score(y_train, y_pred)))
print(confusion_matrix(y_train, y_pred))
print(classification_report(y_train, y_pred))

y_pred = ensemble.predict(X_test)
y_pred2 =  ensemble.predict_proba(X_test)

print('Accuracy of ensemble classifier on test set: {:.2f}'.format(accuracy_score(y_test, y_pred)))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

print(competition_log_loss(y_test, y_pred2))


Accuracy of ensemble classifier on train set: 1.00
[[407   0]
 [  2  84]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       407
           1       1.00      0.98      0.99        86

   micro avg       1.00      1.00      1.00       493
   macro avg       1.00      0.99      0.99       493
weighted avg       1.00      1.00      1.00       493

Accuracy of ensemble classifier on test set: 0.92
[[99  3]
 [ 7 15]]
              precision    recall  f1-score   support

           0       0.93      0.97      0.95       102
           1       0.83      0.68      0.75        22

   micro avg       0.92      0.92      0.92       124
   macro avg       0.88      0.83      0.85       124
weighted avg       0.92      0.92      0.92       124

(124, 2)
(124,) (124,) (124,)
0.44558104598232023


### Test data

In [ ]:
test = pd.read_csv(os.path.join(directory, "test.csv"))
test.head()


,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
0,00eed32682bb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,010ebe33f668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,02fa521e1838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,040e15f562a2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,046e85c7cc7f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# remove BQ and EL since they have more 50 null values
test = test.drop(['BQ', 'EL'], axis=1)

# fill null values with mean
null_cols = test.columns[test.isnull().any()]
for col in null_cols:
    test[col].fillna(test[col].mean(), inplace=True)

# drop Id column
test_id = test['Id']
test = test.drop(['Id'], axis=1)

# make categorical data into numerical data
test["EJ"] = test["EJ"].replace({'A': 0, 'B': 1})
test.head()

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FR,FS,GB,GE,GF,GH,GI,GL,EJ_A,EJ_B
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0


In [ ]:
# predict
y_pred = ensemble.predict(test)
y_prob = ensemble.predict_proba(test)



NameError: name 'pca' is not defined

In [ ]:
out = pd.DataFrame(
    {
        "Id": test_id,
        "class_0": y_prob[:,0],
        "class_1": y_prob[:,1],
    }
)

In [ ]:
out

In [ ]:
out.to_csv(r"submission.csv", index=False)
